## Generate the observations

In [1]:
import numpy as np
import openturns as ot

In [2]:
ot.RandomGenerator.SetSeed(0)

In [3]:
def modelChaboche(X):
    strain,R,C,gamma = X
    stress = R + C*(1-np.exp(-gamma*strain))
    return [stress]

In [4]:
g = ot.PythonFunction(4, 1, modelChaboche) 

In [5]:
Strain = ot.Uniform(0,0.07)
unknownR = 750e6
unknownC = 2750e6
unknownGamma = 10
R = ot.Dirac(unknownR)
C = ot.Dirac(unknownC)
Gamma = ot.Dirac(unknownGamma)

Strain.setDescription(["Strain"])
R.setDescription(["R"])
C.setDescription(["C"])
Gamma.setDescription(["Gamma"])

Create the joint input distribution function.

In [6]:
inputRandomVector = ot.ComposedDistribution([Strain, R, C, Gamma])

Create the Monte-Carlo sample.

In [7]:
size = 100
inputSample = inputRandomVector.getSample(size)
outputStress = g(inputSample)
outputStress[0:5]

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=5 dimension=1 description=[y0] data=[[1.73052e+09],[2.01764e+09],[9.98458e+08],[8.11861e+08],[1.34312e+09]]

Generate observation noise.

In [8]:
stressObservationNoiseSigma = 40.e6 # (Pa)
noiseSigma = ot.Normal(0.,stressObservationNoiseSigma)
sampleNoiseH = noiseSigma.getSample(size)
outputObservations = outputStress + sampleNoiseH

In [9]:
inputObservations = inputSample[:,0]

## Set the calibration parameters

Define the value of the reference values of the $\theta$ parameter. In the bayesian framework, this is called the mean of the *prior* gaussian distribution. In the data assimilation framework, this is called the *background*.

In [10]:
R = 700e6 # Exact : 750e6
C = 2500e6 # Exact : 2750e6
Gamma = 8. # Exact : 10
candidate = ot.Point([R,C,Gamma])

The following statement create the calibrated function from the model. The calibrated parameters Ks, Zv, Zm are at indices 1, 2, 3 in the inputs arguments of the model.

In [11]:
calibratedIndices = [1,2,3]
model = ot.ParametricFunction(g, calibratedIndices, candidate)

## Gaussian linear calibration

The standard deviation of the observations.

In [12]:
sigmaStress = 1.e7 # (Pa)

Define the covariance matrix of the output Y of the model.

In [13]:
errorCovariance = ot.CovarianceMatrix(1)
errorCovariance[0,0] = sigmaStress**2

Defined the covariance matrix of the parameters $\theta$ to calibrate.

In [14]:
sigmaR = 0.1 * R
sigmaC = 0.1 * C
sigmaGamma = 0.1 * Gamma

In [15]:
parameterCovariance = ot.CovarianceMatrix(3)
parameterCovariance[0,0] = sigmaR**2
parameterCovariance[1,1] = sigmaC**2
parameterCovariance[2,2] = sigmaGamma**2
parameterCovariance

class=CovarianceMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[4.9e+15,0,0,0,6.25e+16,0,0,0,0.64]

## Calibration

The `GaussianLinearCalibration` class performs the gaussian linear calibration by linearizing the model in the neighbourhood of the prior.

In [16]:
algo = ot.GaussianLinearCalibration(model, inputObservations, outputObservations, candidate, parameterCovariance, errorCovariance)

The `run` method computes the solution of the problem.

In [17]:
algo.run()

In [18]:
calibrationResult = algo.getResult()

## Analysis of the results

The `getParameterMAP` method returns the maximum of the posterior distribution of $\theta$.

In [19]:
thetaStar = calibrationResult.getParameterMAP()
thetaStar

class=Point name=Unnamed dimension=3 values=[7e+08,2.5e+09,11.6727]

We can compute a 95% confidence interval of the parameter $\theta^\star$. 

In [20]:
thetaPosterior = calibrationResult.getParameterPosterior()
thetaPosterior

class=Normal name=Normal dimension=3 mean=class=Point name=Unnamed dimension=3 values=[7e+08,2.5e+09,11.6727] sigma=class=Point name=Unnamed dimension=3 values=[7e+07,2.5e+08,1.34647] correlationMatrix=class=CorrelationMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[1,-1.98832e-16,-0.677978,-1.98832e-16,1,-0.735007,-0.677978,-0.735007,1]

In [21]:
covarianceThetaStar = thetaPosterior.getCovariance()
covarianceThetaStar

class=CovarianceMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[4.9e+15,-3.47955,-6.39015e+07,-3.47955,6.25e+16,-2.47416e+08,-6.39015e+07,-2.47416e+08,1.81299]

## Calibration based on Kalman matrix

In [22]:
parameterDimension = candidate.getDimension()

In [23]:
model.setParameter(candidate)

In [24]:
modelObservations = model(inputObservations)
modelObservations[0:5]

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=5 dimension=1 description=[y0] data=[[1.44308e+09],[1.67511e+09],[8.82391e+08],[7.45092e+08],[1.14158e+09]]

In [25]:
transposedGradientObservations = ot.Matrix(parameterDimension,size)
for i in range(size):
    g = model.parameterGradient(inputObservations[i])
    transposedGradientObservations[:,i] = g

In [26]:
gradientObservations = transposedGradientObservations.transpose()
gradientObservations[0:5,:]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=5 columns=3 values=[1.00136,1.00136,1.00136,1.00136,1.00136,0.309944,0.393391,0.0715256,0.0178814,0.178814,7.74648e+07,9.42324e+07,2.19462e+07,5.58539e+06,5.00072e+07]

In [27]:
deltay = outputObservations - modelObservations
deltay = deltay.asPoint()
deltay[0:5]

class=Point name=Unnamed dimension=5 values=[2.87598e+08,4.48771e+08,1.26878e+08,8.47323e+07,2.54641e+08]

Compute inverses of B and R.

In [28]:
B = ot.CovarianceMatrix(parameterCovariance)
IB = ot.IdentityMatrix(parameterDimension)
invB = B.solveLinearSystem(IB)

In [29]:
R = ot.CovarianceMatrix(size)
IR = ot.IdentityMatrix(size)
invR = R.solveLinearSystem(IR)

Calcule $A^{-1} = B^{-1} + J^T R^{-1} J = B^{-1} + J^T (J^T R^{-1})^T$.

Soit $C =J^T R^{-1}$. 

Cela implique $A^{-1} = B^{-1} + J^T C^T$.

In [30]:
C = gradientObservations.transpose() * invR

In [31]:
invA = invB + C * gradientObservations
invA

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[100.272,25.1993,6.48398e+09,25.1993,7.82634,1.96823e+09,6.48398e+09,1.96823e+09,4.97039e+17]

Calcule $K = A J^T R^{-1} = A C = (A^{-1})^{-1} C$.

In [32]:
K = invA.solveLinearSystem(C)

Calcule $\hat{\theta} = \mu  + K (y - H(\mu ))$.

In [33]:
thetaStar = candidate + K * deltay
thetaStar

class=Point name=Unnamed dimension=3 values=[7e+08,2.5e+09,11.6739]

Calcule la matrice $A$.

In [34]:
IA = ot.IdentityMatrix(parameterDimension)
A = invA.solveLinearSystem(IA)
A

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[3.42383e-34,1.03931e-34,2.62459e-26,1.03931e-34,3.15486e-35,7.967e-27,2.62459e-26,7.967e-27,2.01192e-18]

Un autre moyen de calculer la matrice de covariance.

In [35]:
L = IB - K * gradientObservations

In [36]:
covarianceThetaStar = K * R * K.transpose() + L * B * L.transpose()
covarianceThetaStar

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[4.9e+15,-3.48174,-6.39216e+07,-3.48174,6.25e+16,-2.47494e+08,-6.39216e+07,-2.47494e+08,1.81393]